In [30]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('train.csv')
# df.info()
df.isna().sum()
df.duplicated().sum()
df.drop_duplicates()
label =  LabelEncoder()
df['model'] = label.fit_transform(df['model'])
df['motor_type'] = label.fit_transform(df['motor_type'])
# df['wheel'] = label.fit_transform(df['wheel'])
df['color'] = label.fit_transform(df['color'])
df['status'] = label.fit_transform(df['status'])
df['type'] = label.fit_transform(df['type'])
def convert_running(value):
    if 'km' in value:
        kilometers = int(value.split()[0])  
        miles = kilometers * 0.621371        
        return f"{miles:.2f} miles"          
    elif 'miles' in value:
        return value  
    else:
        return value 
df['running'] = df['running'].apply(convert_running)
df['running'] = df['running'].str.replace('miles', '')

In [32]:
df.head()

,model,year,motor_type,running,wheel,color,type,status,motor_volume,price
0,4,2022,3,1864.11,left,15,5,1,2.0,24500
1,2,2014,3,82020.97,left,1,5,1,2.0,25500
2,1,2018,3,95000,left,10,5,1,2.0,11700
3,2,2002,3,137000,left,6,5,1,3.2,12000
4,2,2017,3,80778.23,left,1,5,2,2.0,26000


In [34]:
df['wheel'] = df['wheel'].map({'left': 1})

In [36]:
df.head()

,model,year,motor_type,running,wheel,color,type,status,motor_volume,price
0,4,2022,3,1864.11,1,15,5,1,2.0,24500
1,2,2014,3,82020.97,1,1,5,1,2.0,25500
2,1,2018,3,95000,1,10,5,1,2.0,11700
3,2,2002,3,137000,1,6,5,1,3.2,12000
4,2,2017,3,80778.23,1,1,5,2,2.0,26000


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1642 entries, 0 to 1641
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         1642 non-null   int32  
 1   year          1642 non-null   int64  
 2   motor_type    1642 non-null   int32  
 3   running       1642 non-null   object 
 4   wheel         1642 non-null   int64  
 5   color         1642 non-null   int32  
 6   type          1642 non-null   int32  
 7   status        1642 non-null   int32  
 8   motor_volume  1642 non-null   float64
 9   price         1642 non-null   int64  
dtypes: float64(1), int32(5), int64(3), object(1)
memory usage: 96.3+ KB


In [40]:
df['running'] = df['running'].str.strip()
# df['running'] = pd.to_numeric(df['running'], errors='coerce')
# df['running'] = df['running'].fillna(0).astype('int64')

In [44]:
df.corr()['price']

model           0.173485
year            0.638699
motor_type      0.238964
running        -0.513133
wheel                NaN
color          -0.036846
type            0.037789
status         -0.282471
motor_volume    0.000885
price           1.000000
Name: price, dtype: float64

In [46]:
df

,model,year,motor_type,running,wheel,color,type,status,motor_volume,price
0,4,2022,3,1864.11,1,15,5,1,2.0,24500
1,2,2014,3,82020.97,1,1,5,1,2.0,25500
2,1,2018,3,95000,1,10,5,1,2.0,11700
3,2,2002,3,137000,1,6,5,1,3.2,12000
4,2,2017,3,80778.23,1,1,5,2,2.0,26000
...,...,...,...,...,...,...,...,...,...,...
1637,0,2017,3,120000,1,16,5,2,2.0,12400
1638,4,2014,3,105633.07,1,1,5,2,2.0,16500
1639,3,2018,3,68900,1,2,6,2,2.0,19500
1640,3,2019,3,31000,1,1,6,1,2.0,19500


In [50]:
x_train = df.drop(['price'], axis = 1)
y_train = df['price']
x_train.shape, y_train.shape

((1642, 9), (1642,))

In [54]:
from sklearn.preprocessing import StandardScaler
scalar =  StandardScaler()
feature_standardise = scalar.fit_transform(x_train)
feature_standardise

array([[ 1.52639053,  1.09243073,  0.26699337, ..., -0.03413756,
        -0.45353122, -0.1384163 ],
       [ 0.05769246, -0.12234706,  0.26699337, ..., -0.03413756,
        -0.45353122, -0.1384163 ],
       [-0.67665657,  0.48504183,  0.26699337, ..., -0.03413756,
        -0.45353122, -0.1384163 ],
       ...,
       [ 0.79204149,  0.48504183,  0.26699337, ...,  1.08693988,
         0.97583205, -0.1384163 ],
       [ 0.79204149,  0.63688906,  0.26699337, ...,  1.08693988,
        -0.45353122, -0.1384163 ],
       [ 1.52639053,  1.09243073,  0.26699337, ..., -0.03413756,
        -0.45353122, -0.1384163 ]])

In [12]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

param_distributions = {
    'n_estimators': [50, 100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'max_depth': [3, 4, 5, 6, 7, 8],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 4, 7],
    'subsample': [0.5, 0.75, 1.0, 1.5]
}


gbr = GradientBoostingRegressor(random_state=42)

random_search = RandomizedSearchCV(
    estimator=gbr,
    param_distributions=param_distributions,
    n_iter=100,
    scoring='neg_mean_absolute_error',
    cv=5,
    verbose=1,
    random_state=42,
    n_jobs=-1
)


random_search.fit(feature_standardise, y_train)


best_params = random_search.best_params_
print(f'Best parameters: {best_params}')


best_model = random_search.best_estimator_

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best parameters: {'subsample': 0.5, 'n_estimators': 50, 'min_samples_split': 15, 'min_samples_leaf': 4, 'max_depth': 6, 'learning_rate': 0.1}


C:\Users\admin\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
135 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
135 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\admin\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\admin\anaconda3\Lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "C:\Users\admin\anaconda3\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "C:\Users\admin\anaconda3\Lib\site-packag

In [64]:
# from sklearn.model_selection import RandomizedSearchCV
# from xgboost import XGBRegressor
# from sklearn.metrics import mean_squared_error

# # Define the parameter grid for XGBoost
# param_distributions = {
#     'n_estimators': [50, 100, 200, 300],
#     'learning_rate': [0.01, 0.1, 0.2, 0.3],
#     'max_depth': [3, 4, 5, 6, 7, 8],
#     'min_child_weight': [1, 3, 5, 7],  # XGBoost uses 'min_child_weight' instead of 'min_samples_split'
#     'gamma': [0, 0.1, 0.2, 0.3],       # Regularization term for XGBoost
#     'subsample': [0.5, 0.75, 1.0],
#     'colsample_bytree': [0.5, 0.75, 1.0],  # Fraction of features to be randomly sampled for each tree
#     'reg_alpha': [0, 0.01, 0.1, 1],    # L1 regularization term
#     'reg_lambda': [0.01, 0.1, 1],      # L2 regularization term
#     'booster': ['gbtree', 'dart'],      # Including booster types for tuning
#     'tree_method': ['auto', 'exact', 'approx', 'hist']  # Including tree methods
# }

# # Initialize the XGBoost Regressor
# xgbr = XGBRegressor(random_state=42, objective='reg:squarederror')

# # Perform RandomizedSearchCV
# random_search = RandomizedSearchCV(
#     estimator=xgbr,
#     param_distributions=param_distributions,
#     n_iter=100,
#     scoring='neg_mean_absolute_error',
#     cv=5,
#     verbose=1,
#     random_state=42,
#     n_jobs=-1
# )

# # Fit the RandomizedSearchCV
# random_search.fit(feature_standardise, y_train)

# # Get the best parameters
# best_params = random_search.best_params_
# print(f'Best parameters: {best_params}')

# # Get the best model from the random search
# best_model = random_search.best_estimator_

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best parameters: {'tree_method': 'exact', 'subsample': 0.75, 'reg_lambda': 0.01, 'reg_alpha': 1, 'n_estimators': 50, 'min_child_weight': 7, 'max_depth': 4, 'learning_rate': 0.1, 'gamma': 0, 'colsample_bytree': 1.0, 'booster': 'gbtree'}


In [66]:
# xgbr = XGBRegressor(subsample=0.5,reg_lambda=0.01,reg_alpha=0,n_estimators=50,min_child_weight=5,max_depth=5,learning_rate=0.1,gamma=0.1,colsample_bytree=1.0,tree_method='exact',booster='gbtree')
# xgbr.fit(feature_standardise, y_train)

XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=1.0, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.1, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=5, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=50, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [14]:
gbr = GradientBoostingRegressor(subsample=0.5,n_estimators=50,min_samples_split=15,min_samples_leaf=4,max_depth=6,learning_rate=0.1,random_state=23)
gbr.fit(feature_standardise, y_train)

GradientBoostingRegressor(max_depth=6, min_samples_leaf=4, min_samples_split=15,
                          n_estimators=50, random_state=23, subsample=0.5)

In [20]:
from sklearn.metrics import  mean_squared_error, mean_absolute_error, r2_score

y_pred = gbr.predict(feature_standardise)
mse = mean_squared_error(y_train, y_pred)
print(f'Mean Squared Error with Gradient Boosting: {mse:.2f}')
mae = mean_absolute_error(y_train, y_pred)
print(mae)
r2 = r2_score(y_train, y_pred)
print(r2)

Mean Squared Error with Gradient Boosting: 7959291.73
1664.5559299893227
0.8453450271621639


In [68]:
# from sklearn.metrics import  mean_squared_error, mean_absolute_error, r2_score

# y_pred = xgbr.predict(feature_standardise)
# mse = mean_squared_error(y_train, y_pred)
# print(f'Mean Squared Error with Gradient Boosting: {mse:.2f}')
# mae = mean_absolute_error(y_train, y_pred)
# print(mae)
# r2 = r2_score(y_train, y_pred)
# print(r2)
# Mean Squared Error with Gradient Boosting: 8145337.38
# 1718.593005980702
# 0.841729998588562

Mean Squared Error with Gradient Boosting: 8523247.04
1734.6453290932361
0.834386944770813


In [72]:
test = pd.read_csv('test.csv')
from sklearn.preprocessing import LabelEncoder
label =  LabelEncoder()


test['model'] = label.fit_transform(test['model'])
test['motor_type'] = label.fit_transform(test['motor_type'])
test['wheel'] = label.fit_transform(test['wheel'])
test['color'] = label.fit_transform(test['color'])
test['status'] = label.fit_transform(test['status'])
test['type'] = label.fit_transform(test['type'])
def convert_runnings(value):
    if 'km' in value:
        kilometers = int(value.split()[0])  
        miles = kilometers * 0.621371        
        return f"{miles:.2f} miles"          
    elif 'miles' in value:
        return value  
    else:
        return value 

test['running'] = test['running'].apply(convert_runnings)
test['running'] = test['running'].str.replace('miles', '')
x_test = test

In [74]:
x_test = x_test.drop(['Id'], axis = 1)

In [76]:
from sklearn.preprocessing import StandardScaler
scalar =  StandardScaler()
feature_standard = scalar.fit_transform(x_test)
feature_standard

array([[-0.62247891,  0.77117487,  0.1137996 , ..., -0.05892716,
        -0.4470452 , -0.10974538],
       [ 0.8692927 ,  0.31182288,  0.1137996 , ...,  1.45476435,
        -0.4470452 , -0.10974538],
       [-1.36836471,  0.9242922 ,  0.1137996 , ..., -0.05892716,
        -0.4470452 , -0.10974538],
       ...,
       [ 0.1234069 , -0.45376377,  0.1137996 , ..., -0.05892716,
         1.04673998, -0.10974538],
       [-0.62247891,  0.77117487,  0.1137996 , ..., -0.05892716,
         1.04673998, -0.10974538],
       [-1.36836471,  0.31182288,  0.1137996 , ..., -0.05892716,
        -0.4470452 , -0.10974538]])

In [28]:
y_predict = gbr.predict(feature_standard)
y_predict
df_train = pd.DataFrame(y_predict, columns=['price'])
df_train

,price
0,16432.750819
1,16596.120506
2,23730.112193
3,14134.790957
4,5206.791508
...,...
406,29567.767882
407,14130.976549
408,13439.773890
409,16476.302764


In [30]:
y_predict = gbr.predict(feature_standard)
y_predict
df_train = pd.DataFrame(y_predict, columns=['price'])
df_train

,price
0,16432.750819
1,16596.120506
2,23730.112193
3,14134.790957
4,5206.791508
...,...
406,29567.767882
407,14130.976549
408,13439.773890
409,16476.302764


In [78]:
y_predict = xgbr.predict(feature_standard)
y_predict
df_train = pd.DataFrame(y_predict, columns=['price'])
df_train

,price
0,17136.433594
1,16483.181641
2,25961.378906
3,14454.627930
4,7468.285645
...,...
406,30314.640625
407,13720.113281
408,12983.827148
409,16069.256836


In [32]:
combined_df = pd.concat([test['Id'], df_train], axis=1)

combined_df

,Id,price
0,0,16432.750819
1,1,16596.120506
2,2,23730.112193
3,3,14134.790957
4,4,5206.791508
...,...,...
406,406,29567.767882
407,407,14130.976549
408,408,13439.773890
409,409,16476.302764


In [80]:
combined_df = pd.concat([test['Id'], df_train], axis=1)

combined_df

,Id,price
0,0,17136.433594
1,1,16483.181641
2,2,25961.378906
3,3,14454.627930
4,4,7468.285645
...,...,...
406,406,30314.640625
407,407,13720.113281
408,408,12983.827148
409,409,16069.256836


In [83]:
combined_df.to_csv('Car_pred.csv', index=False)

In [34]:
y_test = pd.read_csv('sample_submission.csv')

In [42]:
y_test = y_test.drop(['Id'], axis = 1)
y_test

,price
0,38
1,20549
2,3927
3,5572
4,28598
...,...
406,2514
407,20307
408,27274
409,14422


In [47]:
mse_a = mean_absolute_error(y_test, df_train)

In [51]:
mse_a 

8490.860780149531

In [44]:
mse_a = mean_absolute_error(y_test, df_train)

In [46]:
mse_a

8490.860780149531